In [1]:
pip install pandas numpy tensorflow scipy matplotlib PySR

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 149.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.3/89.3 kB 4.8 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Load your Excel data
df = pd.read_excel('data.xlsx')

# Preview data
print(df.head())

# Extract necessary columns
# Assuming columns are: Patient ID, Time, Group, MRI Feature 1, MRI Feature 2, PET Feature, Misfolded Protein Concentration
time = df['Time'].values
group = df['Group'].values
mri_feature1 = df['MRI Feature 1'].values
mri_feature2 = df['MRI Feature 2'].values
pet_feature = df['PET Feature'].values
misfolded_concentration = df['Misfolded Protein Concentration'].values

# Normalize MRI and PET features
scaler = StandardScaler()
mri_features = scaler.fit_transform(np.column_stack((mri_feature1, mri_feature2)))
pet_features = scaler.fit_transform(pet_feature.reshape(-1, 1))

# Prepare time and misfolded concentration data
# For PINNs, time (t) will be the input, and misfolded protein concentration (c) will be the output


   Subject ID      Time    Group  MRI Feature 1  MRI Feature 2  PET Feature  \
0           1  2.857121  Control      -0.268889       0.399223     0.455888   
1           1  3.748706  Control      -1.106526       0.647196     2.165002   
2           1  5.833688  Control       2.573360      -0.483186    -0.643518   
3           1  7.460449  Control       0.059218       1.573987     0.927840   
4           1  9.621725  Control       0.013929      -1.225766     0.057013   

   Misfolded Protein Concentration  
0                         0.483987  
1                         0.567134  
2                         0.521320  
3                         0.424803  
4                         0.468095  


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define a simple neural network model for c(t)
def create_pinn_model():
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=(1,)))  # Time as input
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(1))  # Output: concentration c(t)
    return model

# Create the model for c(t)
model_c = create_pinn_model()

# Define a model for the reaction term f(c)
def create_reaction_model():
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=(1,)))  # Misfolded protein concentration c as input
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(50, activation='tanh'))
    model.add(layers.Dense(1))  # Output: reaction term f(c)
    return model

# Create the model for f(c)
model_f = create_reaction_model()

# Combine the models into a joint loss function
def pinn_loss(model_c, model_f, t, c_data):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(t)
        c_pred = model_c(t)
        f_pred = model_f(c_pred)
        dc_dt = tape.gradient(c_pred, t)

    # Loss terms
    loss_data = tf.reduce_mean(tf.square(c_pred - c_data))  # Data loss
    loss_residual = tf.reduce_mean(tf.square(dc_dt - f_pred))  # Residuals of the PDE
    total_loss = loss_data + loss_residual
    return total_loss

# Define the optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Training loop (simplified)
for epoch in range(10000):  # You can adjust the number of epochs
    with tf.GradientTape(persistent=True) as tape:
        # Convert time into tensorflow tensor
        t_tensor = tf.convert_to_tensor(time.reshape(-1, 1), dtype=tf.float32)
        c_tensor = tf.convert_to_tensor(misfolded_concentration, dtype=tf.float32)

        # Calculate loss
        loss = pinn_loss(model_c, model_f, t_tensor, c_tensor)

    optimizer_c = tf.keras.optimizers.Adam()  # For model_c
    optimizer_f = tf.keras.optimizers.Adam()  # For model_f

    # Perform gradient descent
    grads_c = tape.gradient(loss, model_c.trainable_variables)
    optimizer_c.apply_gradients(zip(grads_c, model_c.trainable_variables))

    # Apply gradients for model_f (with the new optimizer)
    grads_f = tape.gradient(loss, model_f.trainable_variables)
    optimizer_f.apply_gradients(zip(grads_f, model_f.trainable_variables))

    if epoch % 1000 == 0:
        print(f'Epoch {epoch}, Loss: {loss.numpy()}')


Epoch 0, Loss: 0.5466825366020203
Epoch 1000, Loss: 0.28345414996147156
Epoch 2000, Loss: 0.27875739336013794
Epoch 3000, Loss: 0.2765740752220154
Epoch 4000, Loss: 0.2753537893295288
Epoch 5000, Loss: 0.27460387349128723
Epoch 6000, Loss: 0.27417609095573425
Epoch 7000, Loss: 0.27388057112693787
Epoch 8000, Loss: 0.27368101477622986
Epoch 9000, Loss: 0.2735513746738434


In [ ]:
import pysr

# Extract the predicted f(c) from the PINNs model
f_predictions = model_f.predict(misfolded_concentration.reshape(-1, 1))

# Symbolic regression using PySR
model = pysr.PySRRegressor(
    unary_operators=["sin", "cos", "log", "exp", "sqrt"],
    binary_operators=["+", "-", "*", "/", "**"],
    niterations=1000,  # Adjust the number of iterations
    # This is optional: You can use a limit for complexity or other parameters
)

# Fit symbolic regression model
symbolic_model = model.fit(misfolded_concentration.reshape(-1, 1), f_predictions)

# Get the symbolic expression of f(c)
print("Discovered symbolic expression for f(c):")
print(symbolic_model)


In [ ]:
# Evaluate the performance of the symbolic regression model
from sklearn.metrics import mean_squared_error

# Get predictions from symbolic regression
f_sym_pred = symbolic_model.predict(misfolded_concentration.reshape(-1, 1))

# Calculate the MSE between predicted f(c) and model predictions
mse = mean_squared_error(f_predictions, f_sym_pred)
print(f'Mean Squared Error of symbolic regression predictions: {mse}')
